In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import solve
from scipy.integrate import solve_ivp


In [3]:
f_modul = lambda t,X, A, m, w, gamma: np.array([X[1] - 1 - m*np.cos(w*X[2]) ,
                            (A - X[1]*(1 + np.exp(X[0])))*gamma,1])

Jacf_modul = lambda t,X,A,m,w,gamma: np.array([[0,1,m*w*np.sin(w*X[2])],
                                                [-gamma*X[1]*np.exp(X[0]),-gamma*(1+np.exp(X[0])),0],
                                                [0,0,0]])
s_modul = lambda t,X,A,m,w,gamma : X[1] - 1 - m*np.cos(w*X[2])
GradS_modul = lambda t,X,A,m,w,gamma: np.array([0,1,m*w*np.sin(w*X[2])])

# s_modul2 = lambda t,X, A,m,w,gamma: gamma*(A - X[1]*(1+np.exp(X[0])))
# GradS_modul2 = lambda t,X,A,m,w,gamma: np.array([-gamma*X[1]*np.exp(X[0]),-gamma*(1+np.exp(X[0]))])

ds_T_modul = lambda t,X: 0.0

In [4]:
def big_system(t, Y_M, f, Jacf, dim, A, m, w, gamma):
    # Solving numerically the initial value problem (dy/dt,dM/dt = (f(t,y),Gradf*M) 
    M = Y_M[dim:].reshape((dim, dim), order = 'F')  # Reshape the flat array back into a dim^2 x dim matrix
    dM_dt = Jacf(t,Y_M[:dim],A,m,w,gamma) @ M  # Compute the matrix derivative
    
    return np.concatenate([f(t, Y_M[:dim],A,m,w,gamma),dM_dt.flatten(order = 'F')])

In [5]:
def Newton_orbit(f,s, X_0,T_0,A, m, w, gamma, Gradf,GradS,ds_T, Max_iter, epsilon):
    dim = np.shape(X_0)[0] #The problem dimension

    #________________________________INITIALISATION___________________________________________________________
    k, Xstar_0, Tstar = 0, X_0, T_0
    
    X_by_iter, T_by_iter = np.zeros((Max_iter,dim)),np.zeros((Max_iter))
    Norm_B = Norm_DeltaX = np.zeros((Max_iter))
    Norm_DeltaX[0] = norm_delta_X = 1 # 

    #_____________Newton iteration loop________________
    while norm_delta_X > epsilon  and k < Max_iter:
        print("Iteration", k, "\n")
        print("Norm(Dx) = ", norm_delta_X,"\n")
        X_by_iter[k,:] = Xstar_0
        Norm_DeltaX[k] = norm_delta_X
                
        X_by_iter[k,:] = Xstar_0
        T_by_iter[k] = Tstar


        X_M = np.zeros((dim+dim**2)) #We solve simustanuously d+d*d ODEs
        monodromy = np.eye(dim) #Initialisation of the monodromy matrix

        X_M[:dim] = Xstar_0
        X_M[dim:] = monodromy.flatten(order='F')
        big_sol= solve_ivp(big_system, [0.0,Tstar], X_M,
                            t_eval=[Tstar],
                            args=(f,Gradf,dim,A, m, w, gamma),
                            method='RK45',**{"rtol": 1e-5,"atol":1e-7}) #It's a function of t
        
        Xstar_T = big_sol.y[:dim,-1] #dimension dim+dim*dim
        monodromy = big_sol.y[dim:,-1].reshape(dim,dim, order = "F") #We take M(T)

        # monodromy = monodromy.reshape(dim,dim, order = "F") #Back to the square matrix format
        d = ds_T(Xstar_0, Tstar)
        c = GradS(Tstar,Xstar_0,A,m,w, gamma).reshape(1,-1)
        bb = f(Tstar, Xstar_T, A, m, w, gamma).reshape(-1,1)
        #Concat the whole matrix
        top = np.hstack((monodromy - np.eye(dim), bb))  # Horizontal stacking of A11=M-I and A12=b
        bottom = np.hstack((c,np.array([[d]])))  # Horizontal stacking of A21=c and A22=d
        Mat = np.vstack((top, bottom))  # Vertical stacking of the two rows
        
        #Right hand side concatenation
        B = np.concatenate((Xstar_T - Xstar_0, np.array([s(Tstar,Xstar_0, A,m,w,gamma)])))
        XX = solve(Mat,-B) #Contain Delta_X and Delta_T
        #XX = np.linalg.lstsq(Mat, -B, rcond=None)[0]
        Delta_X = XX[:dim]
        Delta_T = XX[-1]
        
        #Updating
        Xstar_0 += Delta_X
        Tstar += Delta_T

        norm_delta_X = np.linalg.norm(Delta_X)
        Norm_B[k] = np.linalg.norm(B)

        k = k+1
  
    return k, T_by_iter, X_by_iter, Norm_B, Norm_DeltaX, monodromy


In [ ]:
def Newton_orbite_T_known(f, X_0,T,A, m, w, gamma, Jacf, Max_iter, epsilon):
    
    dim = np.shape(X_0)[0] #The problem's dimension
    #________________________________INITIALISATION_____________________________________
    k, Xstar_0 = 0, X_0
    # sol = solve_ivp(lambda t, X: f(t,X, A, m, w, gamma),t_span=[0.0, T], y0=X_0, t_eval=[T],
                    #  method='RK45', option={"rtol": 1e-5,"atol":1e-7})
    

    X_by_iter , Norm_DeltaX, Norm_B = np.zeros((Max_iter+1,dim)),np.zeros((Max_iter+1)),np.zeros((Max_iter+1))
    X_by_iter[0,:] = Xstar_0
    Norm_DeltaX[0] = norm_delta_X = 1 # 
    # Norm_B[0] = norm_B = np.linalg.norm(sol.y[:,-1] - Xstar_0, ord=2)

    #_____________Newton iteration loop____________________
    while (norm_delta_X > epsilon  and k<Max_iter):
        
        #- Solving numerically the initial value problem (dy/dt,dM/dt = (f(t,y),Gradf*M)
        M = np.eye(dim) #Initialisation of the monodromy matrix
        X_M = np.concatenate([Xstar_0, M.flatten()])
        #  Solve the system of ODEs
        big_sol= solve_ivp(lambda t, X: big_system(t, X, f, Jacf, dim, A, m, w, gamma), [0.0,T], y0=X_M,
                            method='RK45',**{"rtol": 1e-10,"atol":1e-12}) #It's a function of t
        
        Xstar_T = big_sol.y[:dim,-1] # The solution at T
        M = big_sol.y[dim:,-1] #We take M(T)
        M = M.reshape((dim,dim), order='F') #Back to the square matrix format
        #______Computation of DeltaX and DeltaT_____
        
        Delta_X = np.linalg.solve(M - np.eye(dim),-(Xstar_T - Xstar_0))

        # Delta_X = np.linalg.lstsq(Mat, -B, rcond=None)[0]
        

        #Updating
        Xstar_0 += Delta_X

        norm_delta_X = np.linalg.norm(Delta_X)
        norm_B = np.linalg.norm(Xstar_T - Xstar_0)

        print("Iteration", k, "\n")
        print("Norm(Dx) = ", norm_delta_X,"\n")
        X_by_iter[k,:] = Xstar_0
        Norm_DeltaX[k] = norm_delta_X
        Norm_B[k] = norm_B
        k += 1
        

    # eigenvalues, eigenvectors = np.linalg.eig(monodromy)
    return k, X_by_iter, Norm_DeltaX,Norm_B, M

In [32]:
Max_iter,epsilon = 100, 1e-12
A = 1.1
m = 2.5e-2
gamma = 1e-3
w = 1e-2
params = (A,m,w,gamma)
L0, D0 = -0.458, .998#-np.log(A-1)+0.001, 1.005 #-5. ,1#0.9805 # m+1
X0 = np.array([L0,D0,461.3])
T0 = 2*np.pi/w

In [ ]:
k, XX,Norm_DeltaX,Norm_B, M = Newton_orbit(f_modul,s_modul, X0,T0,A, m, w, gamma, Jacf_modul,GradS_modul,ds_T_modul, Max_iter, epsilon)


In [ ]:
np.set_printoptions(precision=18)
print(XX[k-1])
k

## Ploting

In [ ]:
fig, ax = plt.subplots(2,2,sharex='all')
Tab = np.asarray(XX[:k])
ax[0,0].plot(np.arange(k),Tab[:,1],'+-')
ax[0,0].set_ylabel(f"$D_0$")

ax[0,1].plot(np.arange(k),Tab[:,0],'+-')
ax[0,1].set_ylabel(f"$L_0$")
ax[1,0].semilogy(np.arange(k),Norm_DeltaX[:k],'x-')
ax[1,0].set_xlabel("Newton iterations")
ax[1,0].set_ylabel(f"$\parallel \Delta X \parallel$")
ax[1,1].semilogy(np.arange(k),Norm_B[:k],'x-')
ax[1,1].set_xlabel("Newton iterations")
ax[1,1].set_ylabel(f"$\parallel \phi(X^*(0),T) - X^*(T) \parallel$")
# ax[1,1].set_ylabel(f"$\parallel (r,s) \parallel_2$")
fig.set_size_inches((8,8))
fig.suptitle(f'Modulated Laser: Fixed point by Newton with fixed period $T=%.3f$ \n $m = %.3f$, $L_0 = %.3f, D_0= %.3f $' % (T0, m, L0,D0))
fig.subplots_adjust(left=0.09, bottom=0.1, right=0.95, top=0.90, hspace=0.35, wspace=0.55)
# plt.savefig(f'./Results/Modulated_Laser_T_known_m_{str(m)}.png')

### Poincaré section

In [ ]:
# visual check #2: Poincare section

#Question: En quoi on est dans une section de Poincaré et dans quelle section est on ?
n_per=1000
npts_par_per=40
t_end=T0*n_per
# 1 point per period
t_eval=np.linspace(0.0,t_end,n_per+1) 


# sol=MyTimestepper.integrate([0,t_end],X0,t_eval=t_eval)

sol = solve_ivp(fun=f_modul,t_span=[0.0, t_end],
                t_eval=t_eval, 
                y0=X0+1e-2, method='RK45', 
                args = params,
                **{"rtol": 1e-5,"atol":1e-7}
                )
print(t_eval.shape,sol.y[0,:].shape)


fig1=plt.figure()
# ax1=fig1.add_subplot(2,1,1)
plt.plot(sol.y[0,:],sol.y[1,:],'.',markersize=5)
plt.title(f'Modulated Laser: Poincare section $T=%.3f$ \n $m = %.3f$, $L_0 = %.3f, D_0= %.3f $' % (T0, m, L0,D0))
plt.xlabel(f"$D$")
plt.ylabel(f"$L$")
plt.savefig(f'./Results/Modulated_Laser_Poincare_Section.png')
plt.tight_layout()

print(t_eval.shape)


## Solving the ODE with the found fixed point 

In [ ]:
X0 = np.array(XX[k-1])
sol = solve_ivp(fun=f_modul,t_span=[0.0, 4*T0],
                t_eval=np.linspace(0.0,4*T0, 1000), 
                y0=X0, method='RK45', 
                args = params,
                **{"rtol": 1e-5,"atol":1e-7}
                )

In [ ]:
X = sol.y 
times_intervall = sol.t
plt.plot(times_intervall,X[0,:])
# plt.xlim([0,1100])
plt.grid()
plt.xlabel("time")
plt.ylabel("L0")
plt.show()

plt.plot(times_intervall,X[1,:])
plt.ylabel("D0")
plt.grid()
plt.show()

plt.plot(X[0,:], X[1,:])
plt.grid()


In [ ]:
eigvalues, eigvec = np.linalg.eig(M)
eigvalues
# Norm_B[k-1]